In [ ]:
import json
import os
from PIL import Image

def process_json_files(path):
    for filename in os.listdir(path):
        if filename.endswith(".json"):
            json_file_path = os.path.join(path, filename)
            with open(json_file_path, 'r') as file:
                data = json.load(file)

            # hc 라벨이 있는지 확인
            hc_labels = [shape for shape in data['shapes'] if shape['label'] == 'hc']
            
            if len(hc_labels):
                hc_labels = [hc_labels[0]]
            
            if hc_labels:
                process_hc_labels(hc_labels, data, path, filename)
                
def process_hc_labels(hc_labels, data, path, json_filename):
    image_path = os.path.join(path, data['imagePath'])
    image = Image.open(image_path)

    # x의 최소, 최대 좌표와 y의 최소 좌표를 구하기
    x_min = min(hc_label['points'][0][0] for hc_label in hc_labels)
    x_max = max(hc_label['points'][1][0] for hc_label in hc_labels)
    y_min = min(hc_label['points'][0][1] for hc_label in hc_labels) - 100

    # 이미지 자르기
    cropped_image = image.crop((0, y_min, image.size[0], image.size[1]))

    # 좌표 수정
    for hc_label in hc_labels:
        hc_label['points'][0][1] -= y_min
        hc_label['points'][1][1] -= y_min
        hc_label['points'][0][0] = 0
        hc_label['points'][1][0] = image.size[0]

    # 새로운 이미지명으로 저장
    new_image_name = f"{os.path.splitext(data['imagePath'])[0]}_1.png"
    
    new_path = "/Users/gwonsmpro/Desktop/hc_test_folder"
    # cropped_image.save(os.path.join(path, new_image_name))
    cropped_image.save(os.path.join(new_path, new_image_name))
    
    # JSON 파일 업데이트
    new_json_data = {
        "version": data["version"],
        "flags": data["flags"],
        "shapes": hc_labels,
        "imagePath": new_image_name,
        "imageData": None,
        "imageWidth": cropped_image.size[0],
        "imageHeight": cropped_image.size[1]
    }

    # 새로운 JSON 파일로 저장
    new_json_file_name = os.path.splitext(json_filename)[0] + "_1.json"
    with open(os.path.join(new_path, new_json_file_name), 'w') as file:
        json.dump(new_json_data, file)

In [ ]:
os_list = [cls for cls in os.listdir("/Users/gwonsmpro/Downloads/pillipse/test_hc/") if os.path.isdir(os.path.join("/Users/gwonsmpro/Downloads/pillipse/test_hc/",cls))]

for cls in os_list:
    process_json_files(f"/Users/gwonsmpro/Downloads/pillipse/test_hc/{cls}")

In [ ]:
import shutil

def process_json_files(path, dest_path):
    for filename in os.listdir(path):
        if filename.endswith(".json"):
            json_file_path = os.path.join(path, filename)
            with open(json_file_path, 'r') as file:
                data = json.load(file)

            # hc 라벨이 있는지 확인
            hc_labels = [shape for shape in data['shapes'] if shape['label'] == 'hc']

            if hc_labels:
                shutil.copy(json_file_path, dest_path)

                # 동일한 이름의 png 파일 복사
                png_filename = os.path.splitext(filename)[0] + ".png"
                png_file_path = os.path.join(path, png_filename)
                if os.path.exists(png_file_path):
                    shutil.copy(png_file_path, dest_path)

base_path = "C:/Users/lee97/OneDrive/바탕 화면/test_hc/sc"
dest_path = "C:/Users/lee97/OneDrive/바탕 화면/test_hc/dest_path"
process_json_files(base_path, dest_path)

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import find_peaks

img = cv2.imread('/Users/gwonsmpro/Documents/GitHub/Utils/testfolder/bcd_0.png', cv2.IMREAD_GRAYSCALE)

projection = np.sum(img, axis=0, dtype=np.int64)
peak_v = np.diff(projection)

first_min = peak_v.argmin()
first_max = peak_v.argmax()

second_smallest_value = np.partition(peak_v.flatten(), 1)[1]
index_second_smallest = np.where(peak_v == second_smallest_value)
second_largest_value = np.partition(peak_v.flatten(), -2)[-2]
index_second_largest = np.where(peak_v == second_largest_value)
# plt.figure(figsize = [5,5])
plt.plot(projection)
plt.vlines(first_min, projection.min(), projection.max(),'red', '--')
plt.vlines(first_max, projection.min(), projection.max(),'blue', '--')
plt.vlines(index_second_smallest, projection.min(), projection.max(),'red', '--')
plt.vlines(index_second_largest, projection.min(), projection.max(),'blue', '--')
plt.show()

# image
# plt.figure(figsize = [5,5])
plt.imshow(img)
plt.show()

projection_h = np.sum(img, axis=1, dtype=np.int64)
peak_h = np.diff(projection_h)

# plt.figure(figsize = [5,5])
plt.plot(projection_h, range(len(projection_h)))
plt.hlines(peak_h.argmin(), projection_h.min(), projection_h.max(), 'red', '--')
plt.hlines(peak_h.argmax(), projection_h.min(), projection_h.max(), 'blue', '--')
plt.gca().invert_yaxis()
plt.show()

In [ ]:

peak_h.argmin()
img = cv2.imread('/Users/gwonsmpro/Documents/GitHub/Utils/testfolder/bcd_4.png')
image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
crop_image = image[peak_h.argmin()-100:peak_h.argmax(), :]


crop_image = cv2.cvtColor(crop_image, cv2.COLOR_BGR2GRAY)

print(crop_image.shape)
crop_image = cv2.resize(crop_image, (crop_image.shape[1],crop_image.shape[0]*10))

# Otsu의 이진화 적용
_, binary_image = cv2.threshold(crop_image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

plt.imshow(crop_image)
plt.show()

plt.imshow(binary_image)
plt.show()

plt.imshow(image[peak_h.argmin()-100:peak_h.argmax(), :])
plt.show()

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import os
import json


base_path = "C:/Users/lee97/OneDrive/바탕 화면/test_hc/compared"

# img_path = "bcd_0.png"
img_list = Path(base_path).glob("*.png")

for img_path in img_list:
    img_path = str(img_path)
    
    img_array = np.fromfile(img_path, np.uint8)
    img = cv2.imdecode(img_array, cv2.IMREAD_COLOR)

    # img = cv2.imread(img_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    projection_h = np.sum(gray[gray.shape[1]//2:, :], axis=1, dtype=np.int64)
    peak_h = np.diff(projection_h)


    mini = min(peak_h.argmin(), peak_h.argmax())
    maxi = max(peak_h.argmin(), peak_h.argmax())

    # print(mini, maxi)

    crop_image_color = image[gray.shape[1]//2 + mini-100:gray.shape[1]//2 + mini+3, :, :]
    crop_image_color = cv2.resize(crop_image_color, (crop_image_color.shape[1], crop_image_color.shape[0]*5))

    # 흰색에 가까운 색상 범위 정의
    lower_white = np.array([170, 170, 170], dtype=np.uint8)
    upper_white = np.array([255, 255, 255], dtype=np.uint8)

    # 흰색에 가까운 픽셀 마스크 생성
    white_mask = cv2.inRange(crop_image_color, lower_white, upper_white)

    # 흰색에 가까운 픽셀만 이진화 (Otsu 적용)
    _, white_binary = cv2.threshold(white_mask, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # 흰색에 가까운 픽셀 이진화 이미지
    # plt.imshow(white_binary, cmap='gray')
    # plt.show()

    # 흰색에 가까운 픽셀을 제외한 나머지 픽셀 이진화 (Otsu 적용)
    non_white_mask = cv2.bitwise_not(white_mask)
    _, non_white_binary = cv2.threshold(non_white_mask, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # 흰색에 가까운 픽셀을 제외한 이진화 이미지
    # plt.imshow(non_white_binary, cmap='gray')
    # plt.show()

    # plt.imshow(crop_image_color)
    # plt.show()

    num_labels, labels, stats, _ = cv2.connectedComponentsWithStats(white_binary, connectivity=8)

    # Bounding Box 및 Labelme JSON 구조 생성
    shapes = []
    for i in range(1, num_labels):  # 배경 레이블 제외
        x, y, w, h, area = stats[i]
        if 100000 > area > 700:  # 노이즈 제거
            y_start = int(mini) - 100 + int(y / 5) + gray.shape[1]//2
            if y_start >= (mini + gray.shape[1]//2 -30):
                y_end = int(mini) - 100 + int((y + h) / 5) + gray.shape[1]//2
                points = [[int(x), y_start], [int(x + w), y_end]]
                shape = {
                    "label": r"hc",
                    "points": points,
                    "group_id": None,
                    "shape_type": "rectangle",
                    "flags": {}
                }
                shapes.append(shape)



    # JSON 형식으로 저장
    json_data = {
        "version": "4.5.6",
        "flags": {},
        "shapes": shapes,
        "lineColor": [0, 255, 0, 128],
        "fillColor": [255, 0, 0, 128],
        "imagePath": os.path.basename(img_path),
        "imageData": None,
        "imageHeight": image.shape[0],
        "imageWidth": image.shape[1],
    }

    json_path = str(Path(img_path).with_suffix(".json"))
    with open(json_path, 'w') as json_file:
        json.dump(json_data, json_file, indent=4)

    # print(f"Bounding boxes saved to {json_path}")


In [ ]:
stats

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path


base_path = "C:/Users/lee97/OneDrive/바탕 화면/test_hc/compared"

img_path = os.path.join(base_path, "bcd_10.png")
# img_list = Path(base_path).glob("*.png")
img_path = "C:/Users/lee97/OneDrive/바탕 화면/0822_labeled_pillip_cropped_hc/train/images/Insp_163518_1.png"
# for img_path in img_list:
img_path = str(img_path)

img_array = np.fromfile(img_path, np.uint8)
img = cv2.imdecode(img_array, cv2.IMREAD_COLOR)

# img = cv2.imread(img_path)
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
projection_h = np.sum(gray[gray.shape[1]//2:, :], axis=1, dtype=np.int64)
peak_h = np.diff(projection_h)


mini = min(peak_h.argmin(), peak_h.argmax())
maxi = max(peak_h.argmin(), peak_h.argmax())

print(mini, maxi)

crop_image_color = image[gray.shape[1]//2 + mini-100:gray.shape[1]//2 + mini+3, :, :]
crop_image_color = cv2.resize(crop_image_color, (crop_image_color.shape[1], crop_image_color.shape[0]*5))

# 흰색에 가까운 색상 범위 정의
lower_white = np.array([170, 170, 170], dtype=np.uint8)
upper_white = np.array([255, 255, 255], dtype=np.uint8)

# 흰색에 가까운 픽셀 마스크 생성
white_mask = cv2.inRange(crop_image_color, lower_white, upper_white)

# 흰색에 가까운 픽셀만 이진화 (Otsu 적용)
_, white_binary = cv2.threshold(white_mask, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

# 흰색에 가까운 픽셀 이진화 이미지
plt.imshow(white_binary, cmap='gray')
plt.show()

# 흰색에 가까운 픽셀을 제외한 나머지 픽셀 이진화 (Otsu 적용)
non_white_mask = cv2.bitwise_not(white_mask)
_, non_white_binary = cv2.threshold(non_white_mask, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

# 흰색에 가까운 픽셀을 제외한 이진화 이미지
plt.imshow(non_white_binary, cmap='gray')
plt.show()

plt.imshow(crop_image_color)
plt.show()

num_labels, labels, stats, _ = cv2.connectedComponentsWithStats(white_binary, connectivity=8)

# Bounding Box 및 Labelme JSON 구조 생성
shapes = []
for i in range(1, num_labels):  # 배경 레이블 제외
    x, y, w, h, area = stats[i]
    if 100000 > area > 700:  # 노이즈 제거
        y_start = int(mini) - 100 + int(y / 5) + gray.shape[1]//2
        if y_start >= (mini + gray.shape[1]//2 -30):
            y_end = int(mini) - 100 + int((y + h) / 5) + gray.shape[1]//2
            points = [[int(x), y_start], [int(x + w), y_end]]
            shape = {
                "label": r"hc",
                "points": points,
                "group_id": None,
                "shape_type": "rectangle",
                "flags": {}
            }
            shapes.append(shape)


# JSON 형식으로 저장
json_data = {
    "version": "4.5.6",
    "flags": {},
    "shapes": shapes,
    "lineColor": [0, 255, 0, 128],
    "fillColor": [255, 0, 0, 128],
    "imagePath": os.path.basename(img_path),
    "imageData": None,
    "imageHeight": image.shape[0],
    "imageWidth": image.shape[1],
}

json_path = str(Path(img_path).with_suffix(".json"))
with open(json_path, 'w') as json_file:
    json.dump(json_data, json_file, indent=4)

print(f"Bounding boxes saved to {json_path}")

for i in range(1, num_labels):  # 배경 레이블 제외
    x, y, w, h, area = stats[i]
    
    if 700 < area < 100000:
        # print(area, y)
        y_start = int(mini) - 100 + int(y / 5) + gray.shape[1]//2
        print(y_start,mini + gray.shape[1]//2)

In [ ]:
import json
from pathlib import Path

def get_iou(box1, box2):
    x1_max = max(box1[0][0], box1[1][0])
    x1_min = min(box1[0][0], box1[1][0])
    y1_max = max(box1[0][1], box1[1][1])
    y1_min = min(box1[0][1], box1[1][1])
    
    x2_max = max(box2[0][0], box2[1][0])
    x2_min = min(box2[0][0], box2[1][0])
    y2_max = max(box2[0][1], box2[1][1])
    y2_min = min(box2[0][1], box2[1][1])

    intersect_x_min = max(x1_min, x2_min)
    intersect_y_min = max(y1_min, y2_min)
    intersect_x_max = min(x1_max, x2_max)
    intersect_y_max = min(y1_max, y2_max)
    
    intersect_area = max(0, intersect_x_max - intersect_x_min + 1) * max(0, intersect_y_max - intersect_y_min + 1)
    
    box1_area = (x1_max - x1_min + 1) * (y1_max - y1_min + 1)
    box2_area = (x2_max - x2_min + 1) * (y2_max - y2_min + 1)
    
    iou = intersect_area / (box1_area + box2_area - intersect_area)
    return iou

compared_path = "c:/Users/lee97/OneDrive/바탕 화면/test_hc/compared"
dest_path ="c:/Users/lee97/OneDrive/바탕 화면/test_hc/dest_path"

compared_json_list = sorted(list(Path(compared_path).glob("*.json"))[:4])
dest_json_list = sorted(list(Path(dest_path).glob("*.json"))[:4])

assert len(compared_json_list) == len(dest_json_list)



for idx, (case1, case2) in enumerate(zip(compared_json_list, dest_json_list)):
    # JSON 데이터 불러오기
    print(idx, case1, case2)
    
    with open(str(case1), "r") as f1:
        case1 = json.load(f1)
    
    with open(str(case2), "r") as f2:
        case2 = json.load(f2)

    # "hc" 레이블을 가진 사각형 추출
    hc_boxes_case1 = [shape["points"] for shape in case1["shapes"] if shape["label"] == "hc"]
    hc_boxes_case2 = [shape["points"] for shape in case2["shapes"] if shape["label"] == "hc"]

    # IoU 계산
    ious = []
    for box1 in hc_boxes_case2:
        for box2 in hc_boxes_case1:
            iou = get_iou(box1, box2)
            ious.append(iou)

    # 결과 출력
    print("IoUs:", ious)
